In [1]:
import pandas as pd

In [2]:
df_fake = pd.read_csv("Fake.csv")

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true = pd.read_csv("True.csv")

In [5]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
df_fake["Class"] = 0
df_true["Class"] = 1

In [7]:
df_fake.duplicated().sum()

3

In [8]:
df_true.duplicated().sum()

206

In [9]:
df_fake.shape

(23481, 5)

In [10]:
df_true.shape

(21417, 5)

In [11]:
df_fake.drop_duplicates(inplace = True)

In [12]:
df_fake.shape

(23478, 5)

In [13]:
df_true.drop_duplicates(inplace = True)

In [14]:
### drop columns (date,title,subject)

In [15]:
df_fake.drop(columns = ["date","title","subject"],inplace = True)

In [16]:
df_fake.head()

,text,Class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [17]:
df_true.drop(columns = ["date","subject","title"],inplace = True)

In [18]:
df_true.head()

,text,Class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [19]:
df = pd.concat([df_fake,df_true],axis = 0)

In [20]:
df.shape

(44689, 2)

In [21]:
df = df.sample(frac = True)

In [22]:
df.head()

,text,Class
16735,BERLIN (Reuters) - The German government has a...,1
5674,WASHINGTON/ANKARA (Reuters) - Instead of teari...,1
17960,"GUIGLO, Ivory Coast (Reuters) - An ethnic-fuel...",1
8641,(Reuters) - A Florida man pleaded guilty on Mo...,1
1437,WASHINGTON (Reuters) - U.S. Defense Secretary ...,1


In [23]:
import re
import string
def regularexp(text):
    text = text.lower()
    text = re.sub("\[.*?\]"," ",text)
    text = re.sub("\\W"," ",text)
    text = re.sub("https?//\S+|www\.\S+"," ",text)
    text = re.sub("<.*?>+"," ",text)
    text = re.sub("[%s]"% re.escape(string.punctuation)," ",text)
    text = re.sub("\n"," ",text)
    text = re.sub("\w*\d\w*"," ",text)
    return text

In [24]:
df["text"] = df["text"].apply(regularexp)

In [25]:
df.head()

,text,Class
16735,berlin reuters the german government has a...,1
5674,washington ankara reuters instead of teari...,1
17960,guiglo ivory coast reuters an ethnic fuel...,1
8641,reuters a florida man pleaded guilty on mo...,1
1437,washington reuters u s defense secretary ...,1


In [26]:
df.reset_index(drop = True,inplace = True)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
tfidf = TfidfVectorizer()

In [33]:
x = df["text"]
y = df["Class"]

In [34]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [35]:
x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)

In [38]:
x_train

<35751x96990 sparse matrix of type '<class 'numpy.float64'>'
	with 7311060 stored elements in Compressed Sparse Row format>

In [39]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [40]:
lr.fit(x_train,y_train)

LogisticRegression()

In [42]:
from sklearn.metrics import accuracy_score
y_pred = lr.predict(x_test)
print("Accuracy_score =",accuracy_score(y_test,y_pred))

Accuracy_score = 0.9864622958156187


In [44]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [45]:
dt.fit(x_train,y_train)

DecisionTreeClassifier()

In [46]:
y_pred = dt.predict(x_test)
print("Accuracy_score =",accuracy_score(y_test,y_pred))

Accuracy_score = 0.996755426269859


In [47]:
from sklearn.ensemble import RandomForestClassifier
rd = RandomForestClassifier()

In [48]:
rd.fit(x_train,y_train)

RandomForestClassifier()

In [49]:
print("Accuracy_score =",accuracy_score(y_test,y_pred))

Accuracy_score = 0.996755426269859


In [50]:
from sklearn.svm import SVC
svc = SVC()

In [51]:
svc.fit(x_train,y_train)

SVC()

In [52]:
y_pred = svc.predict(x_test)

In [53]:
print("Accuracy_score =",accuracy_score(y_test,y_pred))

Accuracy_score = 0.9937346162452451


In [56]:
from sklearn.ensemble import GradientBoostingClassifier
gd = GradientBoostingClassifier()

In [57]:
gd.fit(x_train,y_train)

GradientBoostingClassifier()

In [58]:
y_pred = gd.predict(x_test)
print("Accuracy_score = ",accuracy_score(y_test,y_pred))

Accuracy_score =  0.9954128440366973


In [59]:
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier()

In [60]:
bag.fit(x_train,y_train)

BaggingClassifier()

In [61]:
y_pred = bag.predict(x_test)
print("Accuracy_score =",accuracy_score(y_test,y_pred))

Accuracy_score = 0.99742671738644


In [64]:
import xgboost as xgb
xgb = xgb.XGBClassifier()

In [65]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [66]:
y_pred = xgb.predict(x_test)
print("Accuracy_score =",accuracy_score(y_test,y_pred))

Accuracy_score = 0.9977623629447304


In [69]:
params = [("lr",lr),("dt",dt),("rd",rd),("svc",svc),("gd",gd),("bag",bag),("xgb",xgb)]

In [70]:
from sklearn.ensemble import StackingClassifier
st = StackingClassifier(estimators=params,final_estimator=xgb)

In [71]:
st.fit(x_train,y_train)

StackingClassifier(estimators=[('lr', LogisticRegression()),
                               ('dt', DecisionTreeClassifier()),
                               ('rd', RandomForestClassifier()), ('svc', SVC()),
                               ('gd', GradientBoostingClassifier()),
                               ('bag', BaggingClassifier()),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...))

In [ ]:
y_pred = st.predict(x_test)
print("Accuracy score =",accuracy_score(y_test,y_pred))

Accuracy score = 0.9978742447974939


In [ ]:
import pickle as pkl

In [ ]:
pkl.dump(tfidf, open("tfidf_vectorizer.pkl", "wb"))

In [ ]:
pkl.dump(xgb,open("fake_news_predict.pkl","wb"))